In [4]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import os
# Keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report
from keras.layers import Dense,Dropout,Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical

PATH = os.getcwd()
PATH_condition = PATH+"/data/condition"
PATH_control = PATH+"/data/control"
PATH_scores = PATH+"/data/scores"

df = pd.read_csv(PATH_scores+"/scores.csv",encoding= 'unicode_escape')
df = df.fillna(0)
# print("df",df)
# print("PATH ",PATH_scores)
# dftrain = pd.read_csv(PATH_scores+"/scores.csv",encoding= 'unicode_escape')
# # dftrain = pd.read_csv('coords_500_train_for_validate.csv')
# dftest = pd.read_csv(PATH_scores+"/scores.csv",encoding= 'unicode_escape')
# # dfvalidate = pd.read_csv('coords_500_validate.csv')

# dftrain_s = dftrain.sample(frac=1, random_state=1234)
# dftest_s = dftest.sample(frac=1, random_state=1234)
# # dfvalidate_s = dfvalidate.sample(frac=1, random_state=1234)

# # normal
Y=df[['solution']]
# print("Label",Y)
X= df.drop(['name','solution','ages','edu',], axis=1) # features
# print("Train",X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print("Train",X_train)
print("label_train",y_train)
print("Test",X_test)
print("label_test",y_test)
# y_train = dftrain_s['solution'] # target value
# X_test = dftest_s.drop(['name','solution'], axis=1) # features
# y_test = dftest_s['solution']
# # X_validate = dfvalidate.drop(['name', 'class'], axis=1)
# # y_validate = dfvalidate_s['class']

In [ ]:
X_train.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
vec = label_encoder.fit_transform(y_train)

In [ ]:
Y = to_categorical(vec, num_classes=2)

In [ ]:
print("Y",Y)

In [ ]:
yec = label_encoder.fit_transform(y_test)
Ytest = to_categorical(yec, num_classes=2)

In [ ]:
print("vec",vec)

In [ ]:
print("yec",yec)

In [ ]:
model = Sequential()
model.add(Dense(1250, input_dim = X_train.shape[1], activation = 'relu')) # input layer requires input_dim param
model.add(Dense(1100, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(900, activation = 'relu'))
model.add(Dense(750, activation = 'relu'))
model.add(Dense(600, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(2, activation='sigmoid'))
opt = keras.optimizers.Adam(learning_rate=0.00008)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])


history = model.fit(X_train, Y, validation_split=0.33, epochs = 200, batch_size=32, verbose=1)

In [ ]:
model = Sequential()
model.add(Dense(1250, input_dim = X_train.shape[1], activation = 'relu')) # input layer requires input_dim param
model.add(Dense(1100, activation = 'relu'))
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(900, activation = 'relu'))
model.add(Dense(750, activation = 'relu'))
model.add(Dense(600, activation = 'relu'))
model.add(Dense(500, activation = 'relu'))
model.add(Dense(2, activation='sigmoid'))
opt = keras.optimizers.Adam(learning_rate=0.00008)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y, validation_split=0.1, epochs = 10, batch_size=32, verbose=1)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_train, Y, batch_size=128)
print("test loss, test acc:", results)

In [ ]:
model.summary()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()

In [ ]:
y_pred_class = model.predict(X_test)
pred_x = np.argmax(y_pred_class, axis=1)
y_test_class = np.argmax(Ytest, axis=1)

In [ ]:
pred_x

In [ ]:
from sklearn.metrics import confusion_matrix
classlist = ['depression','non-depression']
P_classlist = ['depression','non-depression']
cm = confusion_matrix(y_test_class, pred_x)

df_cm = pd.DataFrame(cm, classlist, P_classlist) #(cm, range(8), range(8))
sns.set(font_scale=1.4) # for label size
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_class, pred_x))

In [ ]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

target = ['depression','non-depression']
# set plot figure size
fig, c_ax = plt.subplots(1,1, figsize = (12, 8))

# function for scoring roc auc score for multi-class
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)

    for (idx, c_label) in enumerate(target):
        fpr, tpr, thresholds = roc_curve(y_test[:,idx].astype(int), y_pred[:,idx])
        c_ax.plot(fpr, tpr, label = '%s (AUC:%0.2f)'  % (c_label, auc(fpr, tpr)))
    c_ax.plot(fpr, fpr, 'b-', label = 'Random Guessing')
    return roc_auc_score(y_test, y_pred, average=average)


print('ROC AUC score:', multiclass_roc_auc_score(y_test_class, pred_x))

c_ax.legend()
c_ax.set_xlabel('False Positive Rate')
c_ax.set_ylabel('True Positive Rate')
plt.show()